In [68]:
import pandas as pd
import numpy as np
import os
file='./data/voynich.txt'

In [69]:
#df = pd.read_csv(file,header=None, sep='\t',engine='python',names=['image','description'])
df = pd.read_csv(file,header=None,sep='>',names=['heading','text'])
# df[['image','did']] = df.image.str.split("#",expand=True,)


print(len(df))
df.head(3)

5214


,heading,text
0,<f1r.P1.1;H,fachys.ykal.ar.ataiin.shol.shory.cthres...
1,<f1r.P1.2;H,sory.ckhar.or.y.kair.chtaiin.shar.are.c...
2,<f1r.P1.3;H,syaiir.sheky.or.ykaiin.shod.cthoary.cth...


In [70]:
df['heading'] = df.heading.str.replace('<','')
df['text'] = df.text.str.strip()
df.head(5)

,heading,text
0,f1r.P1.1;H,fachys.ykal.ar.ataiin.shol.shory.cthres.y.kor....
1,f1r.P1.2;H,sory.ckhar.or.y.kair.chtaiin.shar.are.cthar.ct...
2,f1r.P1.3;H,syaiir.sheky.or.ykaiin.shod.cthoary.cthes.dara...
3,f1r.P1.4;H,ooiin.oteey.oteos.roloty.cth*ar.daiin.otaiin.o...
4,f1r.P1.5;H,dair.y.chear.cthaiin.cphar.cfhaiin


In [71]:
df[['page','paragraph','line']] = df.heading.str.split(".",expand=True)

In [72]:
df['page']=df['page'].str.replace('f','')
df['text']=df['text'].str.replace('.',' ')
df['text']=df['text'].str.replace('=',' ')

C:\Users\bakig\AppData\Local\Temp\ipykernel_4868\2797887583.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['text']=df['text'].str.replace('.',' ')


In [73]:
df.head(3)

,heading,text,page,paragraph,line
0,f1r.P1.1;H,fachys ykal ar ataiin shol shory cthres y kor ...,1r,P1,1;H
1,f1r.P1.2;H,sory ckhar or y kair chtaiin shar are cthar ct...,1r,P1,2;H
2,f1r.P1.3;H,syaiir sheky or ykaiin shod cthoary cthes dara...,1r,P1,3;H


In [ ]:
df.to_csv('data.csv')

In [74]:
data=df['text']

Creating our own tokenizer

In [75]:
def get_training_corpus():
    for i in range(0, len(df)):
        yield df.iloc[i : ]["text"]

In [76]:
import re
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers, processors, decoders
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))



In [77]:
tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace((r"[\p{Other}&&[^\n\t\r]]"), ""),
        normalizers.Replace((r"[\s]"), " "),
        normalizers.Replace(("!"), ""),
        normalizers.Replace(("%"), ""),
        normalizers.Lowercase(),
        normalizers.NFD(), normalizers.StripAccents()]
)

In [78]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [79]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [81]:
tokenizer.decoder = decoders.WordPiece(prefix="##",cleanup=True)

In [82]:
tokenizer.save('token_voynich1.json')

In [7]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [182]:
df.to_csv('df.csv')

In [ ]:

txt_files_dir='./data/tokenfiles/'

In [63]:
#Group by paragraph
datadf=df.groupby(['page','paragraph'])['text'].apply(lambda x: ''.join(x)).reset_index()

#group by page
#datadf=df.groupby(['page'])['text'].apply(lambda x: ''.join(x)).reset_index()

In [65]:
datadf['size']=datadf['text'].str.len()

In [66]:
datadf.sort_values(by=['page'],inplace=True)


In [67]:
data=datadf['text']
datadf

,page,paragraph,text,size
0,100r,L1,chosarosholsochorcfhyotearchofarysar char-daiindy,49
1,100r,L2,osarochalsainsoitysosamdakocthsofal,35
2,100r,L3,okeeosshockheyorololcheomokolsoteol,35
3,100r,P1,pcheol sheod qocpheeckhy shodol cthdaoto ch qe...,238
4,100r,P2,folshody chol daiin fchodycheol cphol qotees s...,360
...,...,...,...,...
467,99v,P1,sol cheols ockhey qockhy qkoldy s-ok oleees ot...,200
469,99v,P3,qoteeoy chokol qokeeo dy qokeeol olpchey doiir...,486
471,9r,T,ytchas oraiin chkar,19
470,9r,P,tydlo choly cthor orchey s shy odaiin sary-sho...,465


In [ ]:
# Store values in a dataframe column (Series object) to files, one file per record
def column_to_files(column, prefix, txt_files_dir):
    # The prefix is a unique ID to avoid to overwrite a text file
    i=prefix
    #For every value in the df, with just one column
    for row in column.to_list():
      # Create the filename using the prefix ID
      file_name = os.path.join(txt_files_dir, str(i)+'.txt')
      try:
        # Create the file and write the column text to it
        f = open(file_name, 'wb')
        f.write(row.encode('utf-8'))
        f.close()
      except Exception as e:  #catch exceptions(for eg. empty rows)
        print(row, e) 
      i+=1
    # Return the last ID
    return i
# Get the training data

# Removing the end of line character \n
data = data.replace("\n"," ")
# Set the ID to 0
prefix=0
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
# Print the last ID
print(prefix)
# Get the test data


In [88]:
from tokenizers import ByteLevelBPETokenizer,CharBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path




paths = [str(x) for x in Path(".").glob("./data/tokenfiles/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)


tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace((r"[\p{Other}&&[^\n\t\r]]"), ""),
        normalizers.Replace((r"[\s]"), " "),
        normalizers.Replace(("!"), ""),
        normalizers.Replace(("%"), ""),
        normalizers.Replace(("*"), ""),
        normalizers.Lowercase(),
        normalizers.NFD(), normalizers.StripAccents()]
)


# Customize training
tokenizer.train(files=paths, vocab_size=60000, min_frequency=1,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])
#Save the Tokenizer to disk
tokenizer.save_model('./data/')

    


['./data/vocab.json', './data/merges.txt']

In [62]:
data

0      chosarosholsochorcfhyotearchofarysar char-daii...
1      tolchdcholsopchorrolsysol eesosykchochdyykchdy...
2      pcheol cheol ol shey qockhol shor yteol sheock...
3      sairalyotaldyotalytadokororarotorarotalasoraly...
4      dordodorolalydardshotodeeodorpolaiin shocthy q...
                             ...                        
220    psheoas sheeor qoepsheody odar ocpheo-opar ysa...
221    okoramogokarydararokysalooro-ainokorsalolskeea...
222    otaramyotoldyotor-chyoldydar-aryotalyolsyarolo...
223    tydlo choly cthor orchey s shy odaiin sary-sho...
224    fochor oporody opy shor daiin qopchypcho qofol...
Name: text, Length: 225, dtype: object

In [47]:

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True,min_len=3)
        final.append(new)
    return (final)

data_words = gen_words(list(data))

print (data_words[0][0:20])

['fachys', 'ykal', 'ataiin', 'shol', 'shory', 'cthres', 'kor', 'sholdy']


In [48]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
ataiin


In [49]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7,
                                           random_state=100,
                                           update_every=1,
                                           #chunksize=50,
                                           passes=50,
                                           iterations=150,
                                           alpha="auto")

In [50]:
lda_model.print_topics()

[(0,
  '0.038*"qotedy" + 0.028*"chckhey" + 0.022*"oly" + 0.018*"okam" + 0.018*"dchedy" + 0.017*"olchedy" + 0.016*"chcphy" + 0.015*"ldy" + 0.014*"oaiin" + 0.014*"opchdy"'),
 (1,
  '0.059*"qokain" + 0.056*"daiin" + 0.053*"chey" + 0.040*"qokaiin" + 0.028*"chol" + 0.023*"qokey" + 0.023*"otain" + 0.022*"otal" + 0.021*"qol" + 0.020*"sheey"'),
 (2,
  '0.052*"okeey" + 0.048*"okaiin" + 0.027*"okar" + 0.026*"chckhy" + 0.025*"okeedy" + 0.024*"okal" + 0.022*"qotaiin" + 0.019*"qokal" + 0.019*"olkeey" + 0.018*"lkain"'),
 (3,
  '0.060*"ain" + 0.057*"otaiin" + 0.052*"oteey" + 0.048*"raiin" + 0.044*"oteedy" + 0.031*"cheo" + 0.029*"dair" + 0.021*"oty" + 0.018*"qotchedy" + 0.018*"shar"'),
 (4,
  '0.086*"chedy" + 0.061*"shey" + 0.061*"qokeey" + 0.057*"qokeedy" + 0.048*"shedy" + 0.025*"qokedy" + 0.024*"otedy" + 0.022*"lchedy" + 0.017*"qoky" + 0.017*"saiin"'),
 (5,
  '0.083*"aiin" + 0.035*"okain" + 0.026*"qokar" + 0.023*"dar" + 0.019*"chdy" + 0.017*"lkeey" + 0.016*"qotain" + 0.014*"lkaiin" + 0.014*"qotar" +

In [52]:
qq=data[1]

In [53]:
datadf.iloc[224,]

page                                                   9v
text    fochor oporody opy shor daiin qopchypcho qofol...
size                                                  475
Name: 224, dtype: object

In [60]:
doctopics=[]
datadf
k=0
for a in data:

    bow = id2word.doc2bow(a.split())
    t = lda_model.get_document_topics(bow,per_word_topics=False,minimum_probability=0.33)
    t.append(datadf.iloc[k,0])
    k=k+1
    doctopics.append(t)
    


In [61]:
doctopics

[['100r'],
 ['100v'],
 ['101r1'],
 ['101v2'],
 ['102r1'],
 ['102r2'],
 ['102v1'],
 ['102v2'],
 [(4, 0.34753382), '103r'],
 ['103v'],
 ['104r'],
 ['104v'],
 ['105r'],
 ['105v'],
 ['106r'],
 ['106v'],
 ['107r'],
 ['107v'],
 ['108r'],
 ['108v'],
 ['10r'],
 [(1, 0.34336203), '10v'],
 ['111r'],
 ['111v'],
 ['112r'],
 ['112v'],
 ['113r'],
 ['113v'],
 ['114r'],
 ['114v'],
 ['115r'],
 ['115v'],
 ['116r'],
 ['11r'],
 ['11v'],
 ['13r'],
 ['13v'],
 ['14r'],
 ['14v'],
 ['15r'],
 ['15v'],
 ['16r'],
 [(1, 0.36583865), '16v'],
 ['17r'],
 ['17v'],
 ['18r'],
 ['18v'],
 ['19r'],
 [(1, 0.34915885), '19v'],
 ['1r'],
 ['1v'],
 ['20r'],
 ['20v'],
 ['21r'],
 ['21v'],
 ['22r'],
 ['22v'],
 ['23r'],
 ['23v'],
 ['24r'],
 ['24v'],
 ['25r'],
 [(1, 0.33740517), '25v'],
 ['26r'],
 ['26v'],
 ['27r'],
 ['27v'],
 ['28r'],
 [(1, 0.34578797), '28v'],
 ['29r'],
 ['29v'],
 ['2r'],
 ['2v'],
 ['30r'],
 ['30v'],
 ['31r'],
 ['31v'],
 ['32r'],
 [(1, 0.33555812), '32v'],
 ['33r'],
 ['33v'],
 ['34r'],
 ['34v'],
 ['35r'],
 ['35v']

In [56]:
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds="mmds", R=20)
vis

c:\_projects\final_project_challenge\fpenv\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.104943 -0.343308       1        1  24.397018
5     -0.011087  0.334629       2        1  20.517052
4     -0.303561  0.148953       3        1  19.285278
2     -0.258305 -0.190601       4        1  15.459125
3      0.231734  0.157835       5        1   8.099081
6      0.233507 -0.063660       6        1   7.844470
0      0.002768 -0.043848       7        1   4.397975, topic_info=         Term        Freq       Total Category  logprob  loglift
1444    chedy  577.000000  577.000000  Default  20.0000  20.0000
324      aiin  590.000000  590.000000  Default  19.0000  19.0000
1364   qokain  497.000000  497.000000  Default  18.0000  18.0000
86       shey  413.000000  413.000000  Default  17.0000  17.0000
299    qokeey  407.000000  407.000000  Default  16.0000  16.0000
...       ...         ...         ...      ...      ...      ...
69      cthey   16.358407   16.965152   Topic7  -4.5385   3.0876
2183   chotar   16.008335   16.623443   Topic7  -4.5601   3.0863
1413    chain   13.930232   14.543882   Topic7  -4.6991   3.0809
2691     oteo   12.521571   13.134525   Topic7  -4.8058   3.0762
940   chckhhy   11.118094   11.746436   Topic7  -4.9246   3.0690

[160 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1842      4  0.990008   aiiin
324       2  0.999394    aiin
1046      5  0.996731     ain
1828      3  0.995609     air
1413      7  0.962604   chain
...     ...       ...     ...
214       6  0.986150     sor
472       5  0.984378   taiin
495       5  0.979851     tar
1168      1  0.988938    teey
1451      7  0.952219  ykeedy

[140 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 5, 3, 4, 7, 1])